# make_geojson.ipynb

### 概要

宇都宮市の[オープンソースライブラリ](https://data.city.utsunomiya.tochigi.jp/)から、[防災データ](https://catalog.city.utsunomiya.tochigi.jp/dataset/shiteikinkyuuhinanbashoichiran)を用いて公開するgeojsonファイルを作成します

In [22]:
# 一時避難場所(令和3年8月現在)
# 2022年6月14日現在このサイトは閉じられています。
# tmp_evacuation_path='https://www.city.utsunomiya.tochigi.jp/_res/projects/default_project/_page_/001/009/937/ichizihinanbasyo0608.csv'
tmp_evacuation_path='./tmp_park.csv'
# 広域避難場所（令和３年８月現在）
# 2022年6月14日現在このサイトは閉じられています。
# wide_area_evacuation_path='https://www.city.utsunomiya.tochigi.jp/_res/projects/default_project/_page_/001/009/938/kouikihinanbasyo.csv'
wide_area_evacuation_path='./wide_park.csv'

In [23]:
import pandas as pd # データフレーム
# import requests # csvへのリクエスト
import io # ファイル名の管理
import re # 緯度と経度の抽出

#### csvデータの読み取り

In [24]:
# 広域避難場所
# r_wide = requests.get(wide_area_evacuation_path)
# df_wide = pd.read_csv(io.BytesIO(r_wide.content),sep=",",encoding='utf-8_sig')
df_wide=pd.read_csv(wide_area_evacuation_path, sep=',', encoding='shift-jis',index_col=0)

In [25]:
# 一時避難場所
# r_tmp = requests.get(tmp_evacuation_path)
# df_tmp = pd.read_csv(io.BytesIO(r_tmp.content),sep=",",encoding='shift-jis')
df_tmp=pd.read_csv(tmp_evacuation_path, sep=',',encoding='shift-jis',index_col=0)

In [26]:
tmp_park_df = df_tmp[df_tmp['名称'].str.contains('公園')]
tmp_park_df

,名称,住所,電話番号,座標
9,錦中央公園（一時避難場所）,今泉町157,028-663-1611（スポーツ振興財団）,139.9024669:36.5657864
11,御幸公園（一時避難場所）,平出工業団地13-1,028-663-1611（スポーツ振興財団）,139.9337652:36.5803203
24,文化会館及び明保野公園（一時避難場所）,明保野町7-66 外,028-636-2121（文化会館）,139.8713327:36.5436572
29,陽南第1公園（一時避難場所）,大塚町1,028-663-1611（スポーツ振興財団）,139.8656817:36.5321054
44,宇都宮城址公園（一時避難場所）,本丸町、旭1丁目地内（市役所東側）,028-632-2529（市公園管理課）,139.8886906:36.5513311


In [27]:
# tmp_park_df.to_csv('tmp_park.csv',encoding='shift-jis')

In [28]:
wide_park_df = df_wide[df_wide['名称'].str.contains('公園')]
wide_park_df

,名称,住所,電話番号,座標
1,ブレックスアリーナ宇都宮〔市体育館〕・駅東公園（広域避難場所）,元今泉5丁目6-18 他,028-663-1611（スポーツ振興財団）,139.9127068:36.5581267
3,八幡山公園（広域避難場所）,塙田５丁目2-70,028-632-2529（市公園管理課）,139.8899724:36.5676559
4,平出工業団地公園（広域避難場所）,平出工業団地30-1,028-662-5813（緑の相談所）,139.9389889:36.5677532
6,宮原運動公園（広域避難場所）,陽南４丁目5-6,028-658-1052,139.8717313:36.5350165
7,栃木県中央公園（広域避難場所）,睦町2-50,028-636-1491（公園管理事務所）,139.8663676:36.5553989


In [29]:
# wide_park_df.to_csv('wide_park.csv',encoding='shift-jis')

#### geojsonの作成

In [30]:
# https://python-geojson.readthedocs.io/en/latest/#featurecollection
from geojson import Point, Feature, FeatureCollection
import folium
from folium.plugins import MarkerCluster # 地図を出力するためのモジュール

In [32]:
geojson_list=[]
for val in wide_park_df.iterrows():
	for idx, col in enumerate(wide_park_df.columns):
		if col == '座標':
			latitude = re.sub(':.*(\d+)','',val[1][idx])
			longitude = re.sub('.*(\d+):','',val[1][idx])
			points = Point((float(longitude), float(latitude)))
		else:
			print(col, '=', val[1][idx])
	print('\n')

latitude = '35.584047' 
longitude = '139.665936'
altitude = '14.000000'
# my_point = Point((float(longitude), float(latitude), float(altitude)))
my_point = Point((float(longitude), float(latitude)))
my_feature = Feature(geometry=my_point)
my_feature_collection = FeatureCollection(my_feature)

名称 = ブレックスアリーナ宇都宮〔市体育館〕・駅東公園（広域避難場所）
住所 = 元今泉5丁目6-18　他
電話番号 = 028-663-1611（スポーツ振興財団）


名称 = 八幡山公園（広域避難場所）
住所 = 塙田５丁目2-70
電話番号 = 028-632-2529（市公園管理課）


名称 = 平出工業団地公園（広域避難場所）
住所 = 平出工業団地30-1
電話番号 = 028-662-5813（緑の相談所）


名称 = 宮原運動公園（広域避難場所）
住所 = 陽南４丁目5-6
電話番号 = 028-658-1052


名称 = 栃木県中央公園（広域避難場所）
住所 = 睦町2-50
電話番号 = 028-636-1491（公園管理事務所）




## ちょっと可視化

In [35]:
def visualize_locations(
                        # df, # データフレーム
                        # longitude_column_name, # データフレームの中で緯度を保存したカラム名
                        # latitude_column_name, # データフレームの中で経度を保存したカラム名
                        zoom=15, # ズームの大きさ、デフォルトで4
                        # idx_range = 15000 # 出力する行数の上限
                        ) -> folium.folium.Map: # Foliumのマップ型を返す
    """
    EXPLAIN : 日本を拡大した地図に、pandasデータフレームのlatitudeおよびlongitudeカラムをプロットする。
    ARGS :
    % df, # データフレーム
    % longitude_column_name, # データフレームの中で緯度を保存したカラム名
    % latitude_column_name, # データフレームの中で経度を保存したカラム名
    % zoom=4, # ズームの大きさ、デフォルトで4
    idx_range = 15000 # 出力する行数の上限
    """

    # 図の大きさを指定する。
    f = folium.Figure(width=1000, height=500)

    # 初期表示の中心の座標を指定して地図を作成する。
    #北緯36度33分55.6秒 東経139度53分0.7秒
    center_lat=36.5590232
    center_lon=139.8966108

    # foliumのMap形のコンストラクタに初期の座標とズーム情報を渡す, fは図の大きさとして指定
    m = folium.Map([center_lat,center_lon], zoom_start=zoom).add_to(f)
    marker_cluster = MarkerCluster().add_to(m)

    cordinate_column_name='座標'

    # データフレームの全ての行のマーカーを作成する。
    for idx in range(df_wide.shape[0]):
        # 緯度
        latitude_num=re.sub(r'\:.*?','',df_wide[cordinate_column_name][idx])
        # 経度
        longitude_num=re.sub(r'.*?\:','',df_wide[cordinate_column_name][idx])
        folium.Marker(location=[latitude_num,longitude_num]).add_to(marker_cluster)

    return m

In [44]:
cordinate_column_name='座標'
for val in df_wide[cordinate_column_name]:
	print(val)
# for val in df_wide:
# 	# 緯度
# 	latitude_num=re.sub(r'\:.*?','',val)
# 	# 経度
# 	longitude_num=re.sub(r'.*?\:','',val)
# 	folium.Marker(location=[latitude_num,longitude_num]).add_to(marker_cluster)


139.9127068:36.5581267
139.8899724:36.5676559
139.9389889:36.5677532
139.8717313:36.5350165
139.8663676:36.5553989


In [36]:
visualize_locations()

KeyError: 0